<a href="https://colab.research.google.com/github/daisyKim12/Tensorflow_Study/blob/main/Lecture_C3_cat_and_dog_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Category 3

Image Classification using CNN

Computer Vision with CNNs

For this exercise you will build a cats v dogs classifier\
using the Cats v Dogs dataset from TFDS.\
Be sure to use the final layer as shown\
**(Dense, 2 neurons, softmax activation)**\
The testing infrastructre will **resize all images to 224x224**\
with **3 bytes of color depth**. Make sure your input layer trains


In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf

from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

#Load Dataset

In [2]:
dataset_name = 'cats_vs_dogs'

train_dataset = tfds.load(name=dataset_name, split='train[:80%]')
valid_dataset = tfds.load(name=dataset_name, split='train[80%:]')

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incomplete56XHMR/cats_vs_dogs-train.tfrecord*...:   0%|…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


In [3]:
for data in train_dataset.take(3):
  # Normalization
  # Resize Image to (224x224)
  # image(x), label(y)
  x = data['image'] / 255
  y = data['label']
  x = tf.image.resize(x, size=(224,224))
  print(x.shape)
  print(y)

(224, 224, 3)
tf.Tensor(1, shape=(), dtype=int64)
(224, 224, 3)
tf.Tensor(1, shape=(), dtype=int64)
(224, 224, 3)
tf.Tensor(1, shape=(), dtype=int64)


In [4]:
def preprocess(data):
  x = data['image']
  y = data['label']
  x = x / 255
  x = tf.image.resize(x, size=(224,224))
  return x, y

In [5]:
batch_size = 32

In [6]:
train_data = train_dataset.map(preprocess).batch(batch_size)
valid_data = valid_dataset.map(preprocess).batch(batch_size)


#Modeling

In [22]:
model= Sequential([
    Conv2D(64, (3,3), input_shape=(224,224,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), input_shape=(224,224,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), input_shape=(224,224,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dropout(0.25),
    Dense(512, activation='relu'),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax'),

])

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 222, 222, 64)      1792      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 111, 111, 64)     0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 109, 109, 128)     73856     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 54, 54, 128)      0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 52, 52, 128)       147584    
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 26, 26, 128)     

#Compile

__Caution__: Label data is **Not** One-Hot-Encoded

In [24]:
first_data = next(iter(train_dataset))
label = first_data['label']
print(label)
print(train_dataset.take(1))

tf.Tensor(1, shape=(), dtype=int64)
<_TakeDataset element_spec={'image': TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), 'image/filename': TensorSpec(shape=(), dtype=tf.string, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>


In [25]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

#ModelCheckpoint

In [26]:
checkpoint_path = "my_checkpoint.ckpt"
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             save_weights_only=True,
                             save_best_only=True,
                             monitor='val_loss',
                             verbose=1)

#Fit

In [27]:
model.fit(train_data, validation_data=(valid_data),
          epochs=10,
          callbacks=[checkpoint],
          )

Epoch 1/10
582/582 [==============================] - ETA: 0s - loss: 0.7055 - acc: 0.5048
Epoch 1: val_loss improved from inf to 0.69348, saving model to my_checkpoint.ckpt
582/582 [==============================] - 71s 115ms/step - loss: 0.7055 - acc: 0.5048 - val_loss: 0.6935 - val_acc: 0.4901
Epoch 2/10
582/582 [==============================] - ETA: 0s - loss: 0.6945 - acc: 0.5039
Epoch 2: val_loss did not improve from 0.69348
582/582 [==============================] - 63s 108ms/step - loss: 0.6945 - acc: 0.5039 - val_loss: 0.6942 - val_acc: 0.4901
Epoch 3/10
582/582 [==============================] - ETA: 0s - loss: 0.6898 - acc: 0.5344
Epoch 3: val_loss improved from 0.69348 to 0.68180, saving model to my_checkpoint.ckpt
582/582 [==============================] - 63s 108ms/step - loss: 0.6898 - acc: 0.5344 - val_loss: 0.6818 - val_acc: 0.5782
Epoch 4/10
582/582 [==============================] - ETA: 0s - loss: 0.6685 - acc: 0.5962
Epoch 4: val_loss improved from 0.68180 to 0.63

In [28]:
model.load_weights(checkpoint_path)

In [29]:
model.evaluate(valid_data)

146/146 [==============================] - 11s 73ms/step - loss: 0.5713 - acc: 0.6986


[0.5712710022926331, 0.6986242532730103]